In [1]:
import numpy as np
import scipy.integrate as ig

In [2]:
x, w = ig._quadrature._cached_roots_legendre(3)

In [3]:
x,w

(array([-0.77459667,  0.        ,  0.77459667]),
 array([0.55555556, 0.88888889, 0.55555556]))

In [4]:
def fast_integration(func, x0, x1, x2, m):
    x, weights = ig._quadrature._cached_roots_legendre(3)

    res = 0

    if m < 0:
        mapped_x = (x2[0]-x0[0])/2*(1+x)+x0[0]
        for i in range(3):
            ls = x1[1] + (x2[1] - x1[1]) / (x2[0] - x1[0]) * (mapped_x[i] - x1[0])
            mapped_y = (x2[1]-ls)/2*(1+x)+ls

            res += np.prod([func(mapped_x[i], y)*weights[i]*w for y,w in zip(mapped_y, weights)])
    
    return res

In [5]:
def exp_triangle_integration(func, quad_order, x0, x1, x2, m):
    if m < 0:
        integrand = lambda ys,x: np.array([func(x,y) for y in np.array(ys)])
        inside = lambda xs, n: np.array([ig.fixed_quad(integrand, 
                                                              x1[1] + (x2[1] - x1[1]) / (x2[0] - x1[0]) * (x - x1[0]), 
                                                              x2[1], args=(x,), n=n)[0] for x in np.array(xs)])
        
        return ig.fixed_quad(inside, x0[0], x2[0], n=quad_order, args=(quad_order,))[0]
    else:
        integrand = lambda ys,x: np.array([func(x,y) for y in np.array(ys)])
        inside = lambda xs, n: np.array([ig.fixed_quad(integrand, 
                                                              x2[1], 
                                                              x2[1] + (x1[1] - x2[1]) / (x1[0] - x2[0]) * (x - x2[0]), 
                                                              args=(x,), n=n)[0] for x in np.array(xs)])
        
        return ig.fixed_quad(inside, x2[0], x0[0], n=quad_order, args=(quad_order,))[0]

In [6]:
def fix_exp_triangle_integration(func, quad_order, x0, x1, x2, m):
    if m < 0:
        integrand = lambda ys,x: np.array([func(x,y) for y in np.array(ys)])
        inside = lambda xs, n: np.array([ig.fixed_quad(lambda ys: np.array([func(x,y) for y in np.array(ys)]), 
                                                              x1[1] + (x2[1] - x1[1]) / (x2[0] - x1[0]) * (x - x1[0]), 
                                                              x2[1], args=(x,), n=n)[0] for x in np.array(xs)])
        
        return ig.fixed_quad(lambda xs: np.array([ig.fixed_quad(lambda ys: np.array([func(x,y) for y in np.array(ys)]), 
                                                              x1[1] + (x2[1] - x1[1]) / (x2[0] - x1[0]) * (x - x1[0]), 
                                                              x2[1], n=quad_order)[0] for x in np.array(xs)]), 
                                                              x0[0], x2[0], n=quad_order)[0]
    else:
        integrand = lambda ys,x: np.array([func(x,y) for y in np.array(ys)])
        inside = lambda xs, n: np.array([ig.fixed_quad(integrand, 
                                                              x2[1], 
                                                              x2[1] + (x1[1] - x2[1]) / (x1[0] - x2[0]) * (x - x2[0]), 
                                                              args=(x,), n=n)[0] for x in np.array(xs)])
        
        return ig.fixed_quad(inside, x2[0], x0[0], n=quad_order, args=(quad_order,))[0]

In [7]:
def dumb_func(x, y):
    return x*y**2+x*y

In [8]:
fast_integration(dumb_func, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), -1)

0.2524525182475573

In [9]:
fix_exp_triangle_integration(dumb_func, 3, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), m=-1)

0.2249999999999999

In [10]:
exp_triangle_integration(dumb_func, 3, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), m=-1)

0.2249999999999999

In [11]:
%%timeit
fast_integration(dumb_func, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), -1)

50.4 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit
exp_triangle_integration(dumb_func, 3, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), m=-1)

76.8 µs ± 3.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
%%timeit
fix_exp_triangle_integration(dumb_func, 3, np.array([0, 1]), np.array([0, 0]), np.array([1, 1]), m=-1)

75.9 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
